In [1]:
import os
import argparse
import math
import time

import ast
from copy import copy

import pandas as pd
import numpy as np
from glob import glob
from pyproj import Proj
from shapely.ops import transform

import pyproj
import ultralytics
from ultralytics import YOLO
#ultralytics.checks()
import rioxarray
from shapely.geometry import Point

In [5]:
import concurrent
import concurrent.futures
from concurrent.futures.thread import ThreadPoolExecutor
import cv2

In [10]:
def get_args_parse():
    parser = argparse.ArgumentParser("")    
    parser.add_argument("--processing_naip_dir", default="/work/csr33/images_for_predictions/processed_naip_data", type=str)
    parser.add_argument("--processing_naip_filename", default="processed_naip_data", type=str)
    parser.add_argument("--chunk_id",  type=int)
    parser.add_argument("--tile_dir", default="/work/csr33/images_for_predictions/naip_tiles", type=str)
    parser.add_argument("--tilename_chunks_path", default='/hpc/home/csr33/ast_object_detection/images_for_prediction/tilename_chunks.npz', type=str)

    parser.add_argument("--model_path", default="/work/csr33/object_detection/runs/detect/baseline_train/weights/best.pt", type=str)
    parser.add_argument("--prediction_dir", default="/work/csr33/images_for_predictions/predictions", type=str)
    parser.add_argument("--prediction_filename", default="predictions", type=str)
    parser.add_argument("--imgsz", default=640, type=int)
    parser.add_argument('--img_dir', type=str, default="/work/csr33/images_for_predictions/naip_imgs")
    args = parser.parse_args()
    return args


In [11]:
import sys
sys.argv = ['my_notebook']
args = get_args_parse()
args.chunk_id = 1

In [12]:
tile_names = os.listdir(args.tile_dir)
tile_names = [os.path.splitext(tile_name)[0] for tile_name in tile_names]

In [13]:
#create directory to store results
os.makedirs(args.prediction_dir, exist_ok=True)
# load model
model = YOLO(args.model_path)  # custom trained model 
# load a subset of the tile paths to predict on
tile_paths = np.load(args.tilename_chunks_path)[str(args.chunk_id)]
tile_names = [os.path.splitext(os.path.basename(tile_path))[0] for tile_path in tile_paths]
#intialize dataframes
predict_df = pd.DataFrame({})
merged_df = pd.DataFrame({})

for tile_name in tile_names:
    print(tile_name)
    img_paths = glob(os.path.join(args.img_dir,"*"+tile_name+"*")) #identify the imgs correspondig to a given tile
    tile_path = os.path.join(args.tile_dir, tile_name +".tif") # specify the tile path
    #obtain tile information
    utmx, utmy, utm_proj, tile_band, tile_height, tile_width = tile_dimensions_and_utm_coords(tile_path) #used
    #predict on images
    predict_df_by_tank = predict_process(img_paths, tile_height, tile_width, args)
    #merge neighboring images
    merged_df_by_tank = merge_predicted_bboxes(predict_df_by_tank, dist_limit = 5)
    # calculate utm and lat lon coords
    merged_df_by_tank[["utm_coords","latlon_coords"]] = merged_df_by_tank["bbox_pixel_coords"].apply(\
                                                        lambda box: get_crs_coords(box, utmx, utmy, utm_proj))
    #specify the projection used 
    merged_df_by_tank["utm_proj"] = [utm_proj] * len(merged_df_by_tank)
    
    #update dataframes
    predict_df = pd.concat([predict_df, predict_df_by_tank], ignore_index=True)
    merged_df = pd.concat([merged_df, merged_df_by_tank], ignore_index=True)
    #delete temp dataframe to conserve memory
    del predict_df_by_tank, merged_df_by_tank
    
predict_df.to_file(os.path.join(args.prediction_dir, f"predictions_{args.chunk_id}.geojson"), driver='GeoJSON')
merged_df.to_file(os.path.join(args.prediction_dir, f"merged_predictions_{args.chunk_id}.geojson"), driver='GeoJSON')


m_3308052_se_17_060_20210501


NameError: name 'tile_dimensions_and_utm_coords' is not defined

In [ ]:
tile_paths

m_3308052_se_17_060_20210501



OutOfMemoryError: CUDA out of memory. Tried to allocate 266.00 MiB. GPU 0 has a total capacty of 31.40 GiB of which 106.12 MiB is free. Process 1391294 has 23.89 GiB memory in use. Including non-PyTorch memory, this process has 7.39 GiB memory in use. Of the allocated memory 6.64 GiB is allocated by PyTorch, and 343.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
merged_bools, merged_characteristics, merged_bboxes = merge_algo(trunc_diff_objs_characteristics,
                                                                 trunc_diff_objs_bboxes, distance_limit)

In [41]:
 merged_df_by_tank["bbox_pixel_coords"].apply(lambda box: get_crs_coords(box, utmx, utmy, utm_proj))

,utm_coords,latlon_coords
0,"[446818.95, 3395365.95, 447778.95, 3395365.95]","[-87.55527460131331, 30.689886742888532, -87.5..."
1,"[449375.25, 3394682.5500000003, 452412.15, 339...","[-87.528551119962, 30.683831591277624, -87.496..."
2,"[446283.45, 3389130.1500000004, 446667.45, 338...","[-87.56054080717084, 30.633595455653165, -87.5..."
3,"[448987.35000000003, 3392431.95, 452059.350000...","[-87.53248946454238, 30.663507236118058, -87.5..."
4,"[448990.65, 3392430.45, 452062.65, 3389550.45]","[-87.53245494493166, 30.66349384229714, -87.50..."
5,"[447626.85000000003, 3388437.7500000005, 44935...","[-87.54648764493402, 30.627407377589865, -87.5..."
6,"[447626.55000000005, 3388435.6500000004, 44935...","[-87.54649066868814, 30.627388415436897, -87.5..."
7,"[447626.25, 3388434.1500000004, 449354.25, 338...","[-87.54649372286299, 30.62737486728348, -87.52..."
8,"[446148.45, 3391811.5500000003, 446340.45, 338...","[-87.56208941625928, 30.657784407948782, -87.5..."
9,"[447182.85000000003, 3389281.35, 448526.850000...","[-87.55116352677712, 30.63499990001919, -87.53..."


In [44]:
pd.DataFrame({:class_names
              


['narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'external_floating_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'undefined_object',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'external_floating_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank',
 'narrow_closed_roof_tank']

In [16]:


def merge_algo(characteristics, bboxes, dist_limit): #used
    merge_bools = [False] * len(characteristics)
    for i, (char1, bbox1) in enumerate(zip(characteristics, bboxes)):
        for j, (char2, bbox2) in enumerate(zip(characteristics, bboxes)):
            if j <= i:
                continue
            # Create a new box if a distances is less than disctance limit defined 
            merge_bool = calc_sim(bbox1, bbox2, dist_limit) 
            if merge_bool == True:
            # Create a new box  
                new_box = merge_boxes(bbox1, bbox2)   
                bboxes[i] = new_box
                #delete previous text boxes
                del bboxes[j]
                
                # Create a new text string
                ##chip_name list
                if char1[0] != char2[0]: #if the chip_names are not the same
                    #make chip_names into an array
                    if type(char1[0]) == str: 
                        chip_names_1 = np.array([char1[0]])
                    if type(char2[0]) == str:
                        chip_names_2 = np.array([char2[0]])
                    chip_names = np.concatenate((chip_names_1, chip_names_2), axis=0)
                    chip_names = np.unique(chip_names).tolist()
                else:
                    chip_names = np.unique(char1[0]).tolist()  #if the chip_names are not the same
                
                #get object type 
                if char1[1] != char2[1]:
                    object_type = 'undefined_object'
                object_type = char1[1]
                
                characteristics[i] = [chip_names, object_type, 'Unspecified', '1', '1']
                #delete previous text 
                del characteristics[j]
                
                #return a new boxes and new text string that are close
                merge_bools[i] = True
    return merge_bools, characteristics, bboxes

[[...]]

In [ ]:
utm_proj = get_utm_proj(tile_path)
utm_projection_tile.append(utm_proj)
#specify tile chracteristics
tile_heights.append(tile_height)
tile_widths.append(tile_width)
tile_depths.append(tile_band)
#specify lat lon 
nw_lon, nw_lat = transform_point_utm_to_wgs84(utm_proj, utmx[0], utmy[0])
se_lon, se_lat = transform_point_utm_to_wgs84(utm_proj, utmx[-1], utmy[-1]) 
nw_lon_tile.append(nw_lon) #NW_coordinates #minlon
nw_lat_tile.append(nw_lat) #NW_coordinates #maxlat
se_lon_tile.append(se_lon) #SE_coordinates #maxlon
se_lat_tile.append(se_lat) #SE_coordinates #minlat

for positive_image in positive_images: #iterate over each image affiliated with a given tile
    #tile and chip names
    image_name = os.path.splitext(positive_image)[0]
    image_names.append(image_name) # The index is a six-digit number like '000023'.
    tile_names_by_chip.append(tile_name)
    #row/col indicies 
    y, x = image_name.split("_")[-2:] #name of tif with the extension removed; y=row;x=col
    y = int(y)
    x = int(x)
    row_indicies.append(y)
    col_indicies.append(x)
    #get the pixel coordinates (indexing starting at 0)
    minx = x*item_dim 
    miny = y*item_dim 
    maxx = (x+1)*item_dim - 1
    maxy = (y+1)*item_dim - 1
    if maxx > tile_width:
        maxx = tile_width - 1
    if maxy > tile_height:
        maxy = tile_height - 1
    minx_pixel.append(minx) #NW (max: Top Left) # used for numpy crop
    miny_pixel.append(miny) #NW (max: Top Left) # used for numpy crop
    maxx_pixel.append(maxx) #SE (min: Bottom right) 
    maxy_pixel.append(maxy) #SE (min: Bottom right) 
    #image utm coordinates


In [ ]:
def add_objects(xml_directory, tile_name, obj_class, obj_truncated, obj_difficult, obj_chip_name,
                obj_xmin, obj_ymin, obj_xmax, obj_ymax): #used
    tree = et.parse(os.path.join(xml_directory, tile_name + ".xml"))
    root = tree.getroot() 
    obj = et.Element("object") #add size to xml
    
    name = et.Element("name") #class
    name.text = str(obj_class) 
    obj.insert(0, name)
    
    pose = et.Element("pose") #pose
    pose.text = "Unspecified" 
    obj.insert(1, pose)
    
    truncated = et.Element("truncated")
    truncated.text = str(obj_truncated)
    obj.insert(2, truncated)

    difficult = et.Element("difficult")
    difficult.text = str(obj_difficult)
    obj.insert(3, difficult)
 
    chip_name = et.Element("chip_name")
    chip_name.text = str(obj_chip_name)
    obj.insert(4, chip_name)

    bndbox = et.Element("bndbox") #bounding box
    xmin = et.Element("xmin") #xmin
    xmin.text = str(obj_xmin) 
    bndbox.insert(0, xmin)
    ymin = et.Element("ymin") #ymin
    ymin.text = str(obj_ymin) 
    bndbox.insert(1, ymin)
    xmax = et.Element("xmax") #xmax
    xmax.text = str(obj_xmax) 
    bndbox.insert(2, xmax)
    ymax = et.Element("ymax") #ymax
    ymax.text = str(obj_ymax) 
    bndbox.insert(3, ymax)
    obj.insert(5, bndbox)
    
    root.append(obj)
    tree = et.ElementTree(root)
    et.indent(tree, space="\t", level=0)
    tree.write(os.path.join(xml_directory, tile_name +".xml"))   


def generate_tile_xmls(images_and_xmls_by_tile_dir, tile_dir, xml_folder_name, tiles_xml_dir, item_dim): #used
    """
    Args:
        images_and_xmls_by_tile_dir(str): path to directory containing folders holding annotations and images
        tile_dir(str): path to directory containing tiles
        
    Returns:
        object:
    """
    tile_names = [f for f in os.listdir(images_and_xmls_by_tile_dir) \
                  if len(glob(os.path.join(images_and_xmls_by_tile_dir, f, "**/*.xml"))) > 0]
    for tile_name in tqdm(tile_names):
        tile_name_ext = tile_name + ".tif"
        #get tile dimensions ##replace with information from tile characteristics
        da = rioxarray.open_rasterio(os.path.join(tile_dir, tile_name_ext))
        tile_band, tile_height, tile_width = da.shape[0], da.shape[1], da.shape[2]
        #specify xml paths for each tile
        positive_xml_dir = os.path.join(images_and_xmls_by_tile_dir, tile_name, xml_folder_name)
        #load a list of images/xmls for each tile
        positive_xmls = os.listdir(positive_xml_dir)
                       
        for index, chip_xml in enumerate(positive_xmls): #iterate over positive xmls
            #load each chipped image xml
            tree = et.parse(os.path.join(positive_xml_dir, chip_xml))
            root = tree.getroot()

            #create the tile xml
            if index == 0:
                resolution = root.find('resolution').text
                capture_date = root.find('capture_date').text
                create_tile_xml(tile_name, tiles_xml_dir, resolution, capture_date, tile_width, tile_height, tile_band)

            #identify rows and columns
            chip_name = os.path.splitext(chip_xml)[0]
            y, x = chip_name.split("_")[-2:] #name of tif with the extension removed; y=row;x=col
            # Each chip xml goes from 1 - 512, specify the "0", or the end point of the last xml
            minx = int(x)*item_dim
            miny = int(y)*item_dim

            #add the bounding boxes
            for obj in root.iter('object'):
                xmlbox = obj.find('bndbox')
                obj_xmin = str(minx + int(xmlbox.find('xmin').text))
                obj_xmax = str(minx + int(xmlbox.find('xmax').text))
                obj_ymin = str(miny + int(xmlbox.find('ymin').text))
                obj_ymax = str(miny + int(xmlbox.find('ymax').text))
                # correct bboxes that extend past the bounds of the tile width/height
                if int(obj_xmin) > tile_width:
                    obj_xmin = str(tile_width)
                if int(obj_xmax) > tile_width:
                    obj_xmax = str(tile_width)
                if int(obj_ymin) > tile_height:
                    obj_ymin = str(tile_height)
                if int(obj_ymax) > tile_height:
                    obj_ymax = str(tile_height)
                add_objects(tiles_xml_dir, tile_name, obj.find('name').text, obj.find('truncated').text,
                            obj.find('difficult').text, chip_name, obj_xmin, obj_ymin, obj_xmax, obj_ymax)

                


def calc_sim(bbox1, bbox2, dist_limit): #used
    """Determine the similarity of distances between bboxes to determine whether bboxes should be merged
    Computer a Matrix similarity of distances of the text and object
    Called in merge_algo
    Arg:
    bbox1(list): a list of the (xmin, ymin, xmax, ymax) coordinates for box 1 
    bbox2(list): a list of the (xmin, ymin, xmax, ymax) coordinates for box 2
    dist_list(int): the maximum threshold (pixel distance) to merge bounding boxes
    Returns:
    (bool): to indicate whether the bboxes should be merged 
    """

    # text: ymin, xmin, ymax, xmax
    # obj: ymin, xmin, ymax, xmax
    bbox1_xmin, bbox1_ymin, bbox1_xmax, bbox1_ymax = bbox1
    bbox2_xmin, bbox2_ymin, bbox2_xmax, bbox2_ymax = bbox2
    x_dist = min(abs(bbox2_xmin-bbox1_xmax), abs(bbox2_xmax-bbox1_xmin))
    y_dist = min(abs(bbox2_ymin-bbox1_ymax), abs(bbox2_ymax-bbox1_ymin))
        
    #define distance if one object is inside the other
    if (bbox2_xmin <= bbox1_xmin) and (bbox2_ymin <= bbox1_ymin) and (bbox2_xmax >= bbox1_xmax) and (bbox2_ymax >= bbox1_ymax):
        return True
    elif (bbox1_xmin <= bbox2_xmin) and (bbox1_ymin <= bbox2_ymin) and (bbox1_xmax >= bbox2_xmax) and (bbox1_ymax >= bbox2_ymax):
        return True
    #determine if both bboxes are close to each other in 1d, and equal or smaller length in the other
    elif (x_dist <= dist_limit) and (bbox1_ymin <= bbox2_ymin) and (bbox1_ymax >= bbox2_ymax): #bb1 bigger
        return True
    elif (x_dist <= dist_limit) and (bbox2_ymin <= bbox1_ymin) and (bbox2_ymax >= bbox1_ymax): #bb2 bigger
        return True
    elif (y_dist <= dist_limit) and (bbox1_xmin <= bbox2_xmin) and (bbox1_xmax >= bbox2_xmax): #bb1 bigger
        return True
    elif (y_dist <= dist_limit) and (bbox2_xmin <= bbox1_xmin) and (bbox2_xmax >= bbox1_xmax): #bb2 bigger
        return True
    else: 
        return False


def calculate_diameter(bbox, resolution = 0.6): #used
    """ Calculate the diameter of a given bounding bbox (in Pascal Voc Format) for imagery of a given resolution
    Arg:
    bbox(list): a list of the (xmin, ymin, xmax, ymax) coordinates for box. Utm coordinates are provided as [nw_x_utm, se_y_utm, se_x_utm, nw_y_utm] to conform with Pascal Voc Format.
    resolution(float): the (gsd) resolution of the imagery
    Returns:
    (diameter): the diameter of the bbox of interest
    
                [#minx, #maxy, #maxx, #minx]
    """
    obj_xmin, obj_ymin, obj_xmax, obj_ymax = bbox
    obj_width = obj_xmax - obj_xmin
    obj_height = obj_ymax - obj_ymin
    diameter = min(obj_width, obj_height) * resolution #meter
    return diameter


def calc_dist_from_latlon_pair(coord1, coord2):
    """Given a two coordinate pairs, calculate the haversine distance between the points
    Args:
    coord1(array): numpy array of the lat,lon for coordinate 1
    coord2(array): numpy array of the lat,lon for coordinate 2
    Returns: the distance between coord1 and 2
    """
    c1_in_radians = [radians(_) for _ in coord1]
    c2_in_radians = [radians(_) for _ in coord2]
    dist = haversine_distances([c1_in_radians, c2_in_radians])
    return(dist[1][0])  # return in radians

def diam_from_width_height(poly):
    """Calculate the diameter of a polygon,calcualated as the lesser of the width and the height,
    using the lat/lon coordinates. 
    Args:
    poly(polygon): shapely polygon
    Returns: 
    diameter(float):diameter in meters
    """
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html#:~:text=The%20Haversine%20(or%20great%20circle,the%20longitude%2C%20given%20in%20radians.
    #less favorable method of height estimation
    coords = np.asarray(poly.exterior.coords.xy) #up,left; low,left; low,right; up,right; up,left;
    coords = np.flip(coords,0) #give coords as lat, lon
    width = calc_width_from_latlon_pair(coords[:,0], coords[:,3])  #up,left #up,right
    height = calc_dist_from_latlon_pair(coords[:,0], coords[:,1])  #up,left #low,left
    diameter = min(width, height) * 6371000 #multiply by Earth radius to get meters
    return diameter


def merge_tile_annotations(tile_characteristics, tiles_xml_dir, tiles_xml_list=None, distance_limit=5): #used
    # References:
    # https: // answers.opencv.org / question / 231263 / merging - nearby - rectanglesedited /
    # https://stackoverflow.com/questions/55593506/merge-the-bounding-boxes-near-by-into-one
    # specify tiles_xml_list
    if tiles_xml_list is None: # if tiles_xml_list not provided, specify the tiles xml list
        tiles_xml_list = os.listdir(tiles_xml_dir)
    # lists for geosons/geodatabase
    tile_names = []
    chip_names = []
    object_class = []
    merged_bbox = []
    geometry = []  
    minx_pixels = []
    miny_pixels = []
    maxx_pixels = []
    maxy_pixels = []
    utm_projection = []
    nw_x_utms = []
    nw_y_utms = []
    se_x_utms = []
    se_y_utms = []
    centroid_lons = []
    centroid_lats = []
    nw_lats = []
    nw_lons = []
    se_lats = []
    se_lons = []
    diameter = []
    for tile_xml in tqdm(tiles_xml_list): # iterate over tiles
        # save bboxes and characteristics
        trunc_diff_objs_bboxes = []
        trunc_diff_objs_characteristics = []
        remaining_objs_bboxes = []
        remaining_objs_characteristics = []
        # get tile name / tile xml path
        tile_name = os.path.splitext(tile_xml)[0]
        tile_xml_path = os.path.join(tiles_xml_dir, tile_xml)
        # load tile characteristics
        tile_characteristics_subset = tile_characteristics[tile_characteristics.loc[:,"tile_name"] == tile_name]
        #get utm coords
        tile_utmx_array = np.linspace(float(tile_characteristics_subset["nw_x_utm_tile_coord"].values[0]), 
                                      float(tile_characteristics_subset["se_x_utm_tile_coord"].values[0]),
                                      int(tile_characteristics_subset["tile_width"].values[0]))
        tile_utmy_array = np.linspace(float(tile_characteristics_subset["nw_y_utm_tile_coord"].values[0]),
                                      float(tile_characteristics_subset["se_y_utm_tile_coord"].values[0]),
                                      int(tile_characteristics_subset["tile_height"].values[0]))
        utm_proj = tile_characteristics_subset["utm_projection"].values[0]
        # load each xml
        tree = et.parse(tile_xml_path)
        root = tree.getroot()
        for obj in root.iter('object'):
            xmlbox = obj.find('bndbox') #get the bboxes
            obj_xmin = xmlbox.find('xmin').text
            obj_ymin = xmlbox.find('ymin').text
            obj_xmax = xmlbox.find('xmax').text
            obj_ymax = xmlbox.find('ymax').text
            # get truncated bboxes/characteristics
            if (int(obj.find('difficult').text) == 1) or (int(obj.find('truncated').text) == 1):
                trunc_diff_objs_bboxes.append([obj_xmin, obj_ymin, obj_xmax, obj_ymax])
                trunc_diff_objs_characteristics.append([obj.find('chip_name').text, obj.find('name').text,
                                                        obj.find('pose').text, obj.find('truncated').text,
                                                        obj.find('difficult').text])
            else: #get remaining bboxes/characteristics
                remaining_objs_bboxes.append([obj_xmin, obj_ymin, obj_xmax, obj_ymax])
                remaining_objs_characteristics.append([obj.find('chip_name').text, obj.find('name').text,
                                                       obj.find('pose').text, obj.find('truncated').text,
                                                       obj.find('difficult').text])
        
        # Add merge bboxes
        trunc_diff_objs_bboxes = np.array(trunc_diff_objs_bboxes).astype(np.int32)
        trunc_diff_objs_bboxes = trunc_diff_objs_bboxes.tolist()
        merged_bools, merged_characteristics, merged_bboxes = merge_algo(trunc_diff_objs_characteristics,
                                                                         trunc_diff_objs_bboxes, distance_limit)

        for j, (merged_bool, char, bbox) in enumerate(zip(merged_bools, merged_characteristics, merged_bboxes)):
            tile_names.append(tile_name) 
            chip_names.append(char[0])
            object_class.append(char[1])
            # state whether bbox were merged
            merged_bbox.append(merged_bool)
            # pixel coordinates, 0 indexed
            minx = bbox[0] - 1
            miny = bbox[1] - 1
            maxx = bbox[2] - 1
            maxy = bbox[3] - 1 

            minx_pixels.append(minx)
            miny_pixels.append(miny)
            maxx_pixels.append(maxx)
            maxy_pixels.append(maxy)
            # geospatial data
            utm_projection.append(utm_proj)
            nw_x_utm = tile_utmx_array[minx]
            nw_y_utm = tile_utmy_array[miny]
            se_x_utm = tile_utmx_array[maxx]
            se_y_utm = tile_utmy_array[maxy]
            nw_x_utms.append(nw_x_utm)
            nw_y_utms.append(nw_y_utm)
            se_x_utms.append(se_x_utm)
            se_y_utms.append(se_y_utm)
            nw_lon, nw_lat = transform_point_utm_to_wgs84(utm_proj, nw_x_utm, nw_y_utm)
            se_lon, se_lat = transform_point_utm_to_wgs84(utm_proj, se_x_utm, se_y_utm)
            nw_lons.append(nw_lon)
            nw_lats.append(nw_lat)
            se_lons.append(se_lon)
            se_lats.append(se_lat)
            geometry.append(Polygon([(nw_lon, nw_lat), (nw_lon, se_lat), 
                                     (se_lon, se_lat), (se_lon, nw_lat)]))
            #add centroid
            utmcentroidx=nw_x_utm+(se_x_utm-nw_x_utm)/2
            utmcentroidy=se_y_utm+(nw_y_utm-se_y_utm)/2
            centroid_lon, centroid_lat = transform_point_utm_to_wgs84(utm_proj, utmcentroidx, utmcentroidy)
            centroid_lons.append(centroid_lon)
            centroid_lats.append(centroid_lat)
            #calculate diameter
            diameter.append(calculate_diameter(bbox))
            
        #Add remaining bboxes
        remaining_objs_bboxes = np.array(remaining_objs_bboxes).astype(np.int32)
        remaining_objs_bboxes = remaining_objs_bboxes.tolist()
        for j, (char, bbox) in enumerate(zip(remaining_objs_characteristics, remaining_objs_bboxes)):
            tile_names.append(tile_name)
            chip_names.append(char[0])
            object_class.append(char[1])
            # state whether bbox were merged
            merged_bbox.append(False)
            # pixel coordinates, 0 indexed
            minx = bbox[0] - 1
            miny = bbox[1] - 1
            maxx = bbox[2] - 1
            maxy = bbox[3] - 1
            minx_pixels.append(minx)
            miny_pixels.append(miny)
            maxx_pixels.append(maxx)
            maxy_pixels.append(maxy)
            #geospatial data
            utm_projection.append(utm_proj)
            nw_x_utm = tile_utmx_array[minx]
            nw_y_utm = tile_utmy_array[miny]
            se_x_utm = tile_utmx_array[maxx]
            se_y_utm = tile_utmy_array[maxy]
            nw_x_utms.append(nw_x_utm)
            nw_y_utms.append(nw_y_utm)
            se_x_utms.append(se_x_utm)
            se_y_utms.append(se_y_utm)
            nw_lon, nw_lat = transform_point_utm_to_wgs84(utm_proj, nw_x_utm, nw_y_utm)
            se_lon, se_lat = transform_point_utm_to_wgs84(utm_proj, se_x_utm, se_y_utm)
            nw_lons.append(nw_lon)
            nw_lats.append(nw_lat)
            se_lons.append(se_lon)
            se_lats.append(se_lat)
            geometry.append(Polygon([(nw_lon, nw_lat), (nw_lon, se_lat), (se_lon, se_lat), (se_lon, nw_lat)]))
            #add centroid
            utmcentroidx=nw_x_utm+(se_x_utm-nw_x_utm)/2
            utmcentroidy=se_y_utm+(nw_y_utm-se_y_utm)/2
            centroid_lon, centroid_lat = transform_point_utm_to_wgs84(utm_proj, utmcentroidx, utmcentroidy)
            centroid_lons.append(centroid_lon)
            centroid_lats.append(centroid_lat)
            #calculate diameter
            diameter.append(calculate_diameter(bbox))
            
    #create geodatabase
    gdf = gpd.GeoDataFrame({"object_class": object_class, 'tile_name': tile_names,'image_name': chip_names, 
            "nw_x_pixel_object_coord": minx_pixels, "nw_y_pixel_object_coord": miny_pixels, #min lon/lat
            "se_x_pixel_object_coord": maxx_pixels, "se_y_pixel_object_coord": maxy_pixels, #max lat
            "utm_projection": utm_projection,
            "nw_x_utm_object_coord": nw_x_utms, "nw_y_utm_object_coord": nw_y_utms, #utm min
            "se_x_utm_object_coord": se_x_utms, "se_y_utm_object_coord": se_y_utms, #utm max             
            "nw_lat_object_coord": nw_lats, "nw_lon_object_coord": nw_lons,#min lon/lat
            "se_lat_object_coord": se_lats, "se_lon_object_coord": se_lons, #min lon/lat
            "centroid_lon_object_coord": centroid_lons, "centroid_lat_object_coord": centroid_lats, #centroid
            'geometry': geometry, 'diameter': diameter, 'merged_bbox': merged_bbox})
    return gdf

In [ ]:
# Run inference on an image with YOLOv8n
#!yolo predict model=yolov8.pt source='https://ultralytics.com/images/zidane.jpg'

100% 6.23M/6.23M [00:00<00:00, 77.2MB/s]
Ultralytics YOLOv8.0.145 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients

100% 165k/165k [00:00<00:00, 7.46MB/s]
image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 365.8ms
Speed: 13.7ms preprocess, 365.8ms inference, 431.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict


In [ ]:
#Use stream=True for processing long videos or large datasets to efficiently manage memory. When stream=False, the results for all frames or data points are stored in memory, which can quickly add up and cause out-of-memory errors for large inputs. In contrast, stream=True utilizes a generator, which only keeps the results of the current frame or data point in memory, significantly reducing memory consumption and preventing out-of-memory issues.

# Run batched inference on a list of images
results = model(['im1.jpg', 'im2.jpg'], stream=True)  # return a generator of Results objects

# Process results generator
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://user-images.githubusercontent.com/26833433/212889447-69e5bdf1-5800-4e29-835e-2ed2336dede2.jpg" width="600">